<a href="https://www.kaggle.com/code/meshkatshb/decorators-zero-to-mastery?scriptVersionId=217907045" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Decorators & Metaclasses
In this jupyter notebook, we will examine what decorators are, how they work, and how you can implement it yourself.
## Description
Decorators are a way to modify or extend the behavior of functions or methods without permanently modifying them. They are often used for logging, enforcing access control and permissions, instrumentation and timing, and caching.



# Decorator

## 1. Function Decorators
A decorator is a function that takes another function and extends its behavior without explicitly modifying it.

### Example
As you can see below, we created a decorator named `@my_decorator`.

### Note
There are multiple available decorators which we'll cover in the upcoming chapters.

In [1]:
def my_decorator(func):
    def wrapper():
        print("Something is happening before the function is called.")
        func()
        print("Something is happening after the function is called.")
    return wrapper

@my_decorator
def say_hello():
    print("Hello!")

say_hello()


Something is happening before the function is called.
Hello!
Something is happening after the function is called.


## 2. Decorators with Arguments
To create decorators that accept arguments, you need to define a decorator factory function that returns the actual decorator.



In [2]:
def repeat(num_times):
    def decorator_repeat(func):
        def wrapper(*args, **kwargs):
            for _ in range(num_times):
                func(*args, **kwargs)
        return wrapper
    return decorator_repeat

@repeat(num_times=3)
def say_hello(name):
    print(f"Hello, {name}!")

say_hello("Alice")


Hello, Alice!
Hello, Alice!
Hello, Alice!


## 3. Class Decorators
Decorators can also be applied to classes.

In [3]:
def add_method(cls):
    cls.extra_method = lambda self: "This is an extra method"
    return cls

@add_method
class MyClass:
    def original_method(self):
        return "This is the original method"

obj = MyClass()
print(obj.original_method())  # Output: This is the original method
print(obj.extra_method())     # Output: This is an extra method


This is the original method
This is an extra method


## 4. Advanced Function Decorators

### 4.1. Decorating Functions with Arguments
When decorating functions that take arguments, the inner wrapper function needs to accept `*args` and `**kwargs` to handle any number of positional and keyword arguments.



In [4]:
import time

def timer(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        print(f"Function {func.__name__} took {end_time - start_time:.4f} seconds")
        return result
    return wrapper

@timer
def compute(x, y):
    time.sleep(2)
    return x + y

print(compute(5, 10))


Function compute took 2.0021 seconds
15


### 4.2. Chaining Decorators
Multiple decorators can be applied to a single function. They are applied from the innermost to the outermost.



In [5]:
def uppercase(func):
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        return result.upper()
    return wrapper

def bold(func):
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        return f"**{result}**"
    return wrapper

@bold
@uppercase
def greet(name):
    return f"Hello, {name}"

print(greet("Alice"))


**HELLO, ALICE**


### 4.3. Decorators with Optional Arguments
Decorators with optional arguments require an extra level of nesting to handle both cases: with and without arguments.



In [6]:
def log(prefix="LOG:"):
    def decorator(func):
        def wrapper(*args, **kwargs):
            print(f"{prefix} {func.__name__} called with args={args} kwargs={kwargs}")
            return func(*args, **kwargs)
        return wrapper
    return decorator

@log()
def add(x, y):
    return x + y

@log(prefix="DEBUG:")
def multiply(x, y):
    return x * y

print(add(2, 3))
print(multiply(2, 3))


LOG: add called with args=(2, 3) kwargs={}
5
DEBUG: multiply called with args=(2, 3) kwargs={}
6


### 4.4. Retry Logic
Decorators can implement retry logic to automatically retry a function if it raises an exception.



In [7]:
import time

def retry(retries=3, delay=1):
    def decorator(func):
        def wrapper(*args, **kwargs):
            last_exception = None
            for _ in range(retries):
                try:
                    return func(*args, **kwargs)
                except Exception as e:
                    last_exception = e
                    time.sleep(delay)
            raise last_exception
        return wrapper
    return decorator

@retry(retries=5, delay=2)
def unstable_operation():
    if time.time() % 2 < 1:
        print('Operation Failed')
#         raise ValueError("Operation failed")
    return "Operation succeeded"

print(unstable_operation())


Operation Failed
Operation succeeded


## 5. Existing Decorators
As I said in the [introduction](#note), there are many useful existing decorators in python in which you can simply use them. Here is a review on them:

### 5.1. @staticmethod and @classmethod
These decorators are used to define static methods and class methods within a class.


#### Example: @staticmethod

In [8]:
class MyClass:
    @staticmethod
    def static_method():
        print("This is a static method.")

MyClass.static_method()  # Output: This is a static method.


This is a static method.


#### Example: @classmethod

In [9]:
class MyClass:
    class_variable = "Hello, World!"

    @classmethod
    def class_method(cls):
        print(f"Class variable is: {cls.class_variable}")

MyClass.class_method()  # Output: Class variable is: Hello, World!


Class variable is: Hello, World!


### 5.2. @property
The `@property` decorator is used to define getter, setter, and deleter methods for class attributes.



In [10]:
class MyClass:
    def __init__(self, value):
        self._value = value

    @property
    def value(self):
        return self._value

    @value.setter
    def value(self, new_value):
        if new_value >= 0:
            self._value = new_value
        else:
            raise ValueError("Value must be non-negative")

obj = MyClass(10)
print(obj.value)  # Output: 10
obj.value = 20
print(obj.value)  # Output: 20


10
20


### 5.3. @lru_cache (functools)
The @lru_cache decorator caches the results of expensive function calls and returns the cached result when the same inputs occur again.



In [11]:
from functools import lru_cache

@lru_cache(maxsize=100)
def expensive_computation(n):
    print(f"Computing {n}...")
    return n * n

print(expensive_computation(4))  # Output: Computing 4... 16
print(expensive_computation(4))  # Output: 16 (cached result)


Computing 4...
16
16


### 5.4. @dataclass (dataclasses)
The @dataclass decorator automatically generates special methods like `__init__()`, `__repr__()`, and `__eq__()` for user-defined classes.



In [12]:
from dataclasses import dataclass

@dataclass
class Point:
    x: int
    y: int

point = Point(1, 2)
print(point)  # Output: Point(x=1, y=2)


Point(x=1, y=2)


### 5.5. @contextmanager (contextlib)
The `@contextmanager` decorator simplifies the creation of context managers.



In [13]:
from contextlib import contextmanager

@contextmanager
def managed_resource(name):
    print(f"Acquiring resource: {name}")
    yield name
    print(f"Releasing resource: {name}")

with managed_resource("my_resource") as resource:
    print(f"Using resource: {resource}")


Acquiring resource: my_resource
Using resource: my_resource
Releasing resource: my_resource


### 5.6. @cached_property (functools, Python 3.8+)
The `@cached_property` decorator converts a method with a single self argument into a property that is cached on the instance.



In [14]:
from functools import cached_property

class MyClass:
    def __init__(self, value):
        self._value = value

    @cached_property
    def computed_value(self):
        print("Computing value...")
        return self._value * 2

obj = MyClass(10)
print(obj.computed_value)  # Output: Computing value... 20
print(obj.computed_value)  # Output: 20 (cached result)


Computing value...
20
20


### 5.7. @dataclass with Field Customization
You can customize the behavior of fields in a dataclass using the field function.



In [15]:
from dataclasses import dataclass, field

@dataclass
class InventoryItem:
    name: str
    price: float = field(default=0.0)
    quantity_on_hand: int = field(default=0, metadata={"unit": "units"})

item = InventoryItem(name="widget", price=5.0, quantity_on_hand=100)
print(item)  # Output: InventoryItem(name='widget', price=5.0, quantity_on_hand=100)


InventoryItem(name='widget', price=5.0, quantity_on_hand=100)


### 5.8. @retry (Third-Party Library - retrying)
The @retry decorator from the retrying library can retry a function if it raises an exception.


For this decorator, you need to install third party via pip:
```bash
pip install retrying
```

In [16]:
!pip install retrying

In [17]:
from retrying import retry

@retry(stop_max_attempt_number=3, wait_fixed=2000)
def unstable_operation():
    if time.time() % 2 < 1:
        print("Operation failed, retrying...")
        raise ValueError("Failed")
    return "Operation succeeded"

try:
    print(unstable_operation())
except Exception as e:
    print(e)


Operation succeeded


# Metaclass
Metaclasses are the 'classes of classes,' meaning they define how classes behave. A class is an instance of a metaclass. Metaclasses are powerful tools for creating APIs and frameworks, allowing developers to enforce specific class behaviors.



## 1. Basic: Creating a Metaclass
By default, Python uses `type` as the metaclass. You can create your own metaclass by inheriting from `type`.




In [18]:
class MyMeta(type):
    def __new__(cls, name, bases, dct):
        print(f"Creating class {name}")
        return super().__new__(cls, name, bases, dct)

class MyClass(metaclass=MyMeta):
    def __init__(self, value):
        self.value = value

    def show(self):
        print(self.value)

obj = MyClass(10)
obj.show()


Creating class MyClass
10


## 2. Enforcing Class Behavior
You can use metaclasses to enforce certain behaviors or constraints on classes.



In [19]:
class AttributeNameEnforcerMeta(type):
    def __new__(cls, name, bases, dct):
        for attr_name in dct:
            if not attr_name.startswith('_'):
                raise ValueError(f"Attribute name '{attr_name}' must start with an underscore.")
        return super().__new__(cls, name, bases, dct)

class MyClass(metaclass=AttributeNameEnforcerMeta):
    _value = 42

    def _show(self):
        print(self._value)

# This will raise an error
# class InvalidClass(metaclass=AttributeNameEnforcerMeta):
#     value = 42

obj = MyClass()
obj._show()


42


## 3. Singleton Metaclass
A common use of metaclasses is implementing the Singleton pattern, ensuring a class has only one instance.



In [20]:
class SingletonMeta(type):
    _instances = {}

    def __call__(cls, *args, **kwargs):
        if cls not in cls._instances:
            instance = super().__call__(*args, **kwargs)
            cls._instances[cls] = instance
        return cls._instances[cls]

class Singleton(metaclass=SingletonMeta):
    def __init__(self, value):
        self.value = value

singleton1 = Singleton(1)
singleton2 = Singleton(2)

print(singleton1.value)  # Output: 1
print(singleton2.value)  # Output: 1
print(singleton1 is singleton2)  # Output: True


1
1
True
